## **Import vari e inizializzazione pacchetti**

In [ ]:
pip install textattack

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **NOTE IMPORTANTI**


> la funzione **Augmenter.augment(arg)** utilizza al suo interno **random**, peccato che non si puo settare il seed durante la fase di costruzione dell'oggetto o quando viene richiamata la funzione, questo ha comportato l'import di random per forzare il seed prima dell'esecuzione del codice.



In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit # per il testing
import regex as re

# **DII**


In [53]:
dataset = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Bert_data/ddi2013-type/train.tsv', sep='\t')
print(f"COLONNE:{dataset.columns}")
print(f"DIMENSIONE:{dataset.shape}")

COLONNE:Index(['index', 'sentence', 'label'], dtype='object')
DIMENSIONE:(18779, 3)


In [54]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [56]:
rar=dataset.copy()
rar.drop_duplicates(subset ="sentence", 
                     keep = "last", inplace = True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

COLONNE:Index(['index', 'sentence', 'label'], dtype='object')
DIMENSIONE:(18439, 3)


Filtering del dataset dei dati duplicati e rimozione parti inutili di esso

In [ ]:
rar=dataset.copy()
#print(rar.groupby(['Text']).count().shape)
rar.drop_duplicates(subset ="Text", 
                     keep = "last", inplace = True) 
rar.drop(['Id','ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator','Time'], axis=1, inplace=True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

In [74]:
from textattack.augmentation import WordNetAugmenter
from collections import OrderedDict
random.seed(33)

my_items = list()
number_of_trasformation_to_add=1

start_time = timeit.default_timer()
augmenter = WordNetAugmenter(pct_words_to_swap=1, transformations_per_example=number_of_trasformation_to_add)

for row in rar.head(6).itertuples():
  result=augmenter.augment(row.sentence)
  my_items.append({'index': row.index, 'label': row.label,'sentence':row.sentence})
  
  for i in range(number_of_trasformation_to_add):
    my_items.append({'index': row.index, 'label': row.label,'sentence': re.sub(r'@[^$]*\$', '@DRUG$', result[i])})
  

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=0.7520406489998095
COLONNE:Index(['index', 'sentence', 'label'], dtype='object')
DIMENSIONE:(12, 3)


In [75]:
augmeted_new.head(6).sentence

0    The concomitant intake of @DRUG$ and @DRUG$ does not affect the pharmacokinetics of either alcohol or acamprosate.          
1    The incidental consumption of @DRUG$ and @DRUG$ does not impress the pharmacokinetics of either inebriant or acamprosate.   
2    The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either @DRUG$ or acamprosate.      
3    The accompanying consumption of @DRUG$ and Acamprosate does not impact the pharmacokinetics of either @DRUG$ or acamprosate.
4    The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either alcohol or @DRUG$.          
5    The sequent uptake of @DRUG$ and Acamprosate does not strike the pharmacokinetics of either inebriant or @DRUG$.            
Name: sentence, dtype: object

In [76]:

# import transformations, contraints, and the Augmenter
from textattack.augmentation import EmbeddingAugmenter
#Set the seed or the augmenter
random.seed(33)

my_items = list()
number_of_trasformation_to_add=1

start_time = timeit.default_timer()
augmenter = EmbeddingAugmenter(pct_words_to_swap=1, transformations_per_example=number_of_trasformation_to_add)

for row in rar.head(10).itertuples():
  result=augmenter.augment(row.sentence)
  my_items.append({'index': row.index,'label':row.label,'sentence':row.sentence})

  for i in range(number_of_trasformation_to_add): 
    my_items.append({'index': row.index,'label':row.label,'sentence': re.sub(r'@[^$]*\$', '@DRUG$', result[i])})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
embedded_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{embedded_new.columns}")
print(f"DIMENSIONE:{embedded_new.shape}")

TIME OF EXECUTION=4.7040050290002
COLONNE:Index(['index', 'sentence', 'label'], dtype='object')
DIMENSIONE:(20, 3)


In [78]:
embedded_new.head(10).sentence

0    The concomitant intake of @DRUG$ and @DRUG$ does not affect the pharmacokinetics of either alcohol or acamprosate.            
1    The concomitant consumption of @DRUG$ and @DRUG$ does not impacting the pharmacokinetics of neither drink or acamprosate.     
2    The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either @DRUG$ or acamprosate.        
3    The concomitant admittance of @DRUG$ and Acamprosate does not impacting the pharmacokinetics of neither @DRUG$ or acamprosate.
4    The concomitant intake of @DRUG$ and Acamprosate does not affect the pharmacokinetics of either alcohol or @DRUG$.            
5    The concomitant admittance of @DRUG$ and Acamprosate does not afflict the pharmacokinetics of nor drinking or @DRUG$.         
6    The concomitant intake of alcohol and @DRUG$ does not affect the pharmacokinetics of either @DRUG$ or acamprosate.            
7    The concomitant admittance of drinking and @DRUG$ does not influences t